## Loading dataset 

In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [2]:
! kaggle datasets download -d rahilmehtaucoe2784/fashion

 98% 289M/294M [00:11<00:00, 29.3MB/s]
100% 294M/294M [00:11<00:00, 25.9MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

## installing and importing Libraries 

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 4.2 MB/s 
     |████████████████████████████████| 1.1 MB 59.7 MB/s 
     |████████████████████████████████| 840 kB 56.1 MB/s 
     |████████████████████████████████| 3.4 MB 57.3 MB/s 
     |████████████████████████████████| 1.1 MB 57.9 MB/s 
     |████████████████████████████████| 6.4 MB 58.9 MB/s 
     |████████████████████████████████| 87 kB 8.4 MB/s 
     |████████████████████████████████| 1.2 MB 60.0 MB/s 
     |████████████████████████████████| 234 kB 74.6 MB/s 
     |████████████████████████████████| 120 kB 73.1 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 78.3 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 11.3 MB/s 
     |████████████████████████████████| 352 kB 67.3 MB/s 
     |████████████████████████████████| 47.8 MB 1.1 MB/s 
     |████████████████████████████████| 462 kB 72.7 MB/s 
     |██████████████

In [5]:
import numpy as np
import os

#from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

## Training 

In [7]:
train_image_dir = '/content/final fashion dataset/train image'
train_annotation_dir = '/content/final fashion dataset/train anotation '
validate_image_dir = '/content/final fashion dataset/valid image'
validate_anaotation_dir = '/content/final fashion dataset/valid anotation '
test_image = "/content/final fashion dataset/test image"
test_anotation = "/content/final fashion dataset/test anotation "

In [8]:
spec = model_spec.get('efficientdet_lite4')

In [9]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani", 7:"tshirt", 8:"Kurti", 9:"Top", 10:"saree"})

In [10]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani",7:"tshirt", 8:"Kurti", 9:"Top", 10:"saree"})

In [11]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image, test_anotation, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani",7:"tshirt", 8:"Kurti", 9:"Top", 10:"saree"})

In [14]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

1330
385
177


In [15]:
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=50, 
                               batch_size=5, 
                               train_whole_model=True, 
                               validation_data=validate_dir
                               )

Epoch 1/50
266/266 [==============================] - 177s 466ms/step - det_loss: 1.1021 - cls_loss: 0.8716 - box_loss: 0.0046 - reg_l2_loss: 0.0941 - loss: 1.1962 - learning_rate: 0.0071 - gradient_norm: 3.2266 - val_det_loss: 0.7257 - val_cls_loss: 0.6458 - val_box_loss: 0.0016 - val_reg_l2_loss: 0.0942 - val_loss: 0.8200
Epoch 2/50
266/266 [==============================] - 121s 456ms/step - det_loss: 0.5905 - cls_loss: 0.5012 - box_loss: 0.0018 - reg_l2_loss: 0.0944 - loss: 0.6848 - learning_rate: 0.0062 - gradient_norm: 3.9712 - val_det_loss: 0.5260 - val_cls_loss: 0.4667 - val_box_loss: 0.0012 - val_reg_l2_loss: 0.0945 - val_loss: 0.6205
Epoch 3/50
266/266 [==============================] - 121s 455ms/step - det_loss: 0.4968 - cls_loss: 0.4312 - box_loss: 0.0013 - reg_l2_loss: 0.0946 - loss: 0.5914 - learning_rate: 0.0062 - gradient_norm: 3.7333 - val_det_loss: 0.5307 - val_cls_loss: 0.4909 - val_box_loss: 7.9670e-04 - val_reg_l2_loss: 0.0947 - val_loss: 0.6254
Epoch 4/50
266/266

## Evaluating the model 

In [16]:
model.evaluate(test_dir,batch_size=5)

36/36 [==============================] - 13s 222ms/step



{'AP': 0.8946277,
 'AP50': 0.95845443,
 'AP75': 0.9495593,
 'AP_/Casual Shirt': 0.84457123,
 'AP_/Desgin sherwani': 0.8544264,
 'AP_/Design Kurta': 0.81755704,
 'AP_/Formal Shirt': 0.9378338,
 'AP_/Kurti': 0.9128713,
 'AP_/Plain kurta': 0.9134251,
 'AP_/Plain sherwani': 0.9860891,
 'AP_/Top': 0.8807209,
 'AP_/saree': 0.8891089,
 'AP_/tshirt': 0.9096733,
 'APl': 0.89463,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.94604975,
 'ARm': -1.0,
 'ARmax1': 0.92488706,
 'ARmax10': 0.9456989,
 'ARmax100': 0.94604975,
 'ARs': -1.0}

## Saving Model 

In [17]:
model.export(export_dir='/content/')